## SICP 习题 （2.77）解题总结： 通用型算术运算

SICP 习题 2.77 涉及的内容是通用型算术运算。

虽然题目本身只是问我们为什么Louis在对复数求(magnitude)时会报错，让我们去debug分析一下。

但是，如果你想重现问题，你需要把这个小节涉及的代码都实现了。

有了前面几道大题的经验以后，我们面对这种题目已经没有压力了，先撸一段代码，把书里涉及到的代码都抄进来，尝试运行它就好了。

不过抄写代码之前还是要理解这个小节的内容。这个小节的内容是关于通用运算，这里还是用了算术运算为例，不管是整数，实数还是复数，它们都有加减乘除，完全可以把这部分抽象出来构建一套通用的应用处理所有数的加减乘除。

首先拷贝的是前面习题里面的三角函数代码，这个是因为Calysto Scheme没有帮我们实现三角函数，所以我们把python里的三角函数打包一下：

In [9]:
(import 'math)

(python-exec "
def cos(input_number):
    return math.cos(input_number)
             ")

(python-exec "
def sin(input_number):
    return math.sin(input_number)
             ")

(python-exec "
def atan(input_number):
    return math.atan(input_number)
             ")

接着是大段的书里的代码：

In [10]:
(define (square x) (* x x))

(define (assoc key records)
;;   (display "---------------")(newline)
;;   (display key)(newline)
;;   (display records)(newline)
;;   (cond (((not (null? records)) (begin (display (car records))(newline)))))
  
  
  (cond ((null? records) #f)
	((equal? key (caar records)) (car records))
	(else (assoc key (cdr records)))))

(define (make-table)
  (let ((local-table (list '*table*)))
    (define (lookup key-1 key-2)
      (let ((subtable (assoc key-1 (cdr local-table))))
	(if subtable
	    (let ((record (assoc key-2 (cdr subtable))))
	      (if record
		  (cdr record)
		  (begin
		    (display "record not found: ") (display key-2) (newline)
		    #f
		    )))
	    (begin 
	      (display "subtable not found: ") (display key-1)(newline)
	      #f))))
    (define (insert! key-1 key-2 value)
      (let ((subtable (assoc key-1 (cdr local-table))))
	(if subtable
	    (let ((record (assoc key-2 (cdr subtable))))
	      (if record
		  (set-cdr! record value)
		  (set-cdr! subtable
			    (cons (cons key-2 value)
				  (cdr subtable)))))
	    (set-cdr! local-table
		      (cons (list key-1
				  (cons key-2 value))
			    (cdr local-table)))))
      'ok)
    (define (dispatch m)
      (cond ((eq? m 'lookup-proc) lookup)
	    ((eq? m 'insert-proc!) insert!)
	    (else (error "Unknown operation -- TABLE" m))))
    dispatch))


(define operation-table (make-table))

(define get (operation-table 'lookup-proc))

(define put (operation-table 'insert-proc!))


(define (attach-tag type-tag contents)
  (cons type-tag contents))

(define (type-tag datum)
  (if (pair? datum)
      (car datum)
      (error "Bad tagged datum -- TYPE-TAG" datum)))

(define (contents datum)
  (if (pair? datum)
      (cdr datum)
      (error "Bad tagged datum -- CONTENTS" datum)))



In [11]:
operation-table

#<procedure>

In [12]:
(get 'make 'scheme-number)

subtable not found: make


#f

In [13]:
;; (install-scheme-number-package)

In [14]:
(define (add x y) (apply-generic 'add x y))

(define (sub x y) (apply-generic 'sub x y))

(define (mul x y) (apply-generic 'mul x y))

(define (div x y) (apply-generic 'div x y))

(define (install-scheme-number-package)
  (define (tag x)
    (attach-tag 'scheme-number x))
  (put 'add '(scheme-number scheme-number)
       (lambda (x y) (tag (+ x y))))
  (put 'sub '(scheme-number scheme-number)
       (lambda (x y) (tag (- x y))))
  (put 'mul '(scheme-number scheme-number)
       (lambda (x y) (tag (* x y))))
  (put 'div '(scheme-number scheme-number)
       (lambda (x y) (tag (/ x y))))
  (put 'make 'scheme-number
       (lambda (x) (tag x)))
  'done)

(define (make-scheme-number n)
  ((get 'make 'scheme-number) n))

In [15]:




(define (install-rational-package)
  (define (numer x) (car x))
  (define (denom x) (cdr x))
  (define (make-rat n d)
    (let ((g (gcd n d)))
      (cons (/ n g) (/ d g))))
  (define (add-rat x y)
    (make-rat (+ (* (numer x) (denom y))
		 (* (numer y) (denom x)))
	      (* (denom x) (denom y))))
  (define (sub-rat x y)
    (make-rat (- (* (numer x) (denom y))
		 (* (numer y) (denom x)))
	      (* (denom x) (denom y))))
  (define (mul-rat x y)
    (make-rat (* (numer x) (numer y))
	      (* (denom x) (denom y))))
  (define (div-rat x y)
    (make-rat (* (numer x) (denom y))
	      (* (denom x) (numer y))))

  (define (tag x) (attach-tag 'rational x))
  (put 'add '(rational rational)
       (lambda (x y) (tag (add-rat x y))))
  (put 'sub '(rational rational)
       (lambda (x y) (tag (sub-rat x y))))
  (put 'mul '(rational rational)
       (lambda (x y) (tag (mul-rat x y))))
  (put 'div '(rational rational)
       (lambda (x y) (tag (div-rat x y))))

  (put 'make 'rational
       (lambda (n d) (tag (make-rat n d))))
  'done)

(define (make-rational n d)
  ((get 'make 'rational) n d))


(define (apply-generic op . args)
  (let ((type-tags (map type-tag args)))
    (let ((proc (get op type-tags)))
      (if proc
	  (apply proc (map contents args))
	  (error 
	   "No method for these types -- APPLY-GENERIC"
	   (list op type-tags))))))

(define (real-part z) (apply-generic 'real-part z))
(define (imag-part z) (apply-generic 'imag-part z))
(define (magnitude z) (apply-generic 'magnitude z))
(define (angle z) (apply-generic 'angle z))


(define (install-rectangular-package)
  (define (real-part z) (car z))
  (define (imag-part z) (cdr z))
  (define (make-from-real-imag x y) (cons x y))
  (define (magnitude z)
    (sqrt (+ (square (real-part z))
	     (square (imag-part z)))))
  (define (angle z)
    (atan (/ (imag-part z) (real-part z))))
  (define (make-from-mag-ang r a)
    (cons (* r (cos a)) (* r (sin a))))

  (define (tag x) (attach-tag 'rectangular x))
  (put 'real-part '(rectangular) real-part)
  (put 'imag-part '(rectangular) imag-part)
  (put 'magnitude '(rectangular) magnitude)
  (put 'angle '(rectangular) angle)
  (put 'make-from-real-imag 'rectangular
       (lambda (x y) (tag (make-from-real-imag x y))))
  (put 'make-from-mag-ang 'rectangular
       (lambda (r a) (tag (make-from-mag-ang r a))))
  'done)

(define (install-polar-package)
  (define (magnitude z) (car z))
  (define (angle z) (cdr z))
  (define (make-from-mag-ang r a) (cons r a))
  (define (real-part z)
    (* (magnitude z) (cos (angle z))))
  (define (imag-part z)
    (* (magnitude z) (sing (angle z))))
  (define (make-from-real-imag x y)
    (cons (sqrt (+ (square x) (square y)))
	  (atan (/ y x))))
  (define (tag x) (attach-tag 'polar x))
  (put 'real-part '(polar) real-part)
  (put 'imag-part '(polar) imag-part)
  (put 'magnitude '(polar) magnitude)
  (put 'angle '(polar) angle)
  (put 'make-from-real-imag 'polar
       (lambda (x y) (tag (make-from-real-imag x y))))
  (put 'make-from-mag-ang 'polar
       (lambda (r a) (tag (make-from-mag-ang r a))))
  'done)

(define (install-complex-package)
  (define (make-from-real-imag x y)
    ((get 'make-from-real-imag 'rectangular) x y))
  (define (make-from-mag-ang r a)
    ((get 'make-from-mag-ang 'polar) r a))

  (define (add-complex z1 z2)
    (make-from-real-imag (+ (real-part z1) (real-part z2))
			 (+ (imag-part z1) (imag-part z2))))

  (define (sub-complex z1 z2)
    (make-from-real-imag (- (real-part z1) (real-part z2))
			 (- (imag-part z1) (imag-part z2))))

  (define (mul-complex z1 z2)
    (make-from-mag-ang (* (magnitude z1) (magnitude z2))
		       (+ (angle z1) (angle z2))))

  (define (div-complex z1 z2)
    (make-from-mag-ang (/ (magnitude z1) (magnitude z2))
		       (- (angle z1 ) (angle z2))))

  (define (tag z) (attach-tag 'complex z))



  (put 'add '(complex complex)
       (lambda (z1 z2) (tag (add-complex z1 z2))))
  (put 'sub '(complex complex)
       (lambda (z1 z2) (tag (sub-complex z1 z2))))
  (put 'mul '(complex complex)
       (lambda (z1 z2) (tag (mul-complex z1 z2))))
  (put 'div '(complex complex)
       (lambda (z1 z2) (tag (div-complex z1 z2))))
  (put 'make-from-real-imag 'complex
       (lambda (x y) (tag (make-from-real-imag x y))))
  (put 'make-from-mag-ang 'complex
       (lambda (r a) (tag (make-from-mag-ang r a))))
  'done)

(define (make-complex-from-real-imag x y)
  ((get 'make-from-real-imag 'complex) x y))
(define (make-complex-from-mag-ang r a)
  ((get 'make-from-mag-ang 'complex) r a))

(define (display-complex complex)
 (display "--------------------") (newline)
  (display "real part:") (display (real-part complex)) (newline)
  (display "imag part:") (display (imag-part complex)) (newline)
  (display "magnitude:") (display (magnitude complex)) (newline)
  (display "angle    :") (display (angle complex))(newline)
  (display "--------------------") (newline)
)

(define (start-test-2-77)
  (install-scheme-number-package)
  (install-rational-package)
  (install-rectangular-package)
  (install-polar-package)
  (install-complex-package)
  (display-complex (make-complex-from-real-imag 3 4))
;;   (display-complex z)
  )



接着我们跑一下测试代码看看，会发现程序报错："record not found (complex)"，看这个错误就知道是数据表里没有发现(complex)这东西。

In [16]:
(start-test-2-77)

--------------------
real part:record not found: (complex)



Traceback (most recent call last):
  File "In [16]", line 1, col 1, in 'start-test-2-77'
  File "In [15]", line 164, col 3, in 'display-complex'
  File "In [15]", line 151, col 35, in 'real-part'
  File "In [15]", line 53, col 23, in 'apply-generic'
  File "In [15]", line 45, col 3, in 'let'
  File "In [15]", line 46, col 5, in 'let'
  File "In [15]", line 49, col 4, in 'error'
UnhandledException: string argument expected, got 'Symbol'



这个时候按书上的提示加入以下代码：

In [17]:
  ; the following lines are added for this execise:
  (put 'real-part '(complex) real-part)
  (put 'imag-part '(complex) imag-part)
  (put 'magnitude '(complex) magnitude)
  (put 'angle '(complex) angle)
  ;end of the execise

ok

再跑一遍测试代码，这次就没有问题了。

In [18]:
(start-test-2-77)

--------------------
real part:3
imag part:4
magnitude:5.0
angle    :0.9272952180016122
--------------------


题目问我们为什么，如果你仔细拆开上面的代码进行测试分析，不难发现是因为(complex)的各种操作没有安装，导致执行这些操作，如real-part, magnitude等时在数据表里找不到对应处理数据。

彩蛋：试试scheme-number的操作，然后直接试试对纯数字进行操作：

In [23]:
(add (make-scheme-number 2) (make-scheme-number 3))

(scheme-number . 5)

下面这个会报错，因为我们目前还不支持直接处理纯数字

In [19]:
(add 2 3)


Traceback (most recent call last):
  File "In [19]", line 1, col 1, in 'add'
  File "In [14]", line 1, col 19, in 'apply-generic'
  File "In [15]", line 45, col 20, in 'map'
  File "In [10]", line 63, col 7, in 'error'
UnhandledException: object of type 'int' has no len()

